# GPT Use Case 2: Analyze the product sales

Import libraries and access API

In [ ]:
import openai
import pandas as pd
import json

GPT_MODEL = "gpt-3.5-turbo-0613"
openai.api_key = API_key = 'sk-...'

Import product sales from csv file

In [ ]:
sale = './sales_data/Sales_April_2019.csv'
df = pd.read_csv(sale)
df.dropna(inplace=True)
df = df.head(20)
dict_sale = df.to_dict(orient='dict')

Create prompt template

In [ ]:
instruction = (\
    """You have no previous memories\n"""
    """You are a professional business analyst.\n"""
    """You task is to summarize each product sales.\n"""
    """You should also provide some interesting statistical information about the sales\n"""
    f"""Sales: {dict_sale}\n"""
    """Return the result into JSON schema\n"""
    )

Input prepared prompt to LLM and run GPT

In [ ]:
response = openai.ChatCompletion.create(
    model=GPT_MODEL,
    messages=[
        {'role': 'system', "content": instruction}
    ],
    temperature=0
)

result = response['choices'][0]['message']['content']

Load and post-process GPT response

In [ ]:
json_result = json.loads(result)
print(json_result)

Export csv file

In [ ]:
new_df = pd.DataFrame.from_dict(json_result['summary'])
new_df.to_csv('./sales_report/sales_summary.csv', encoding='utf-8-sig')
print(new_df)